# Advanced Tax Policy Simulation with DSGE Model

このノートブックでは、改良版の線形化手法（Klein法）を使用して、より高度な税制シミュレーションを実行します。

## 主な機能:
- 移行動態の詳細な分析
- 厚生分析（消費等価変分）
- 複数の税制改革シナリオの比較
- 最適税制の探索

In [ ]:
# 必要なライブラリのインポート
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# プロジェクトのルートディレクトリをPythonパスに追加
sys.path.append(os.path.dirname(os.getcwd()))

# 改良版モジュールのインポート
from src.dsge_model import DSGEModel, ModelParameters, load_model
from src.linearization_improved import ImprovedLinearizedDSGE
from src.tax_simulator import EnhancedTaxSimulator, TaxReform

# プロット設定
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.labelsize'] = 12
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['legend.fontsize'] = 10

# 日本語フォントの設定
# plt.rcParams['font.sans-serif'] = ['Hiragino Sans', 'Yu Gothic', 'Meiryo', 'DejaVu Sans']

print("ライブラリのインポートが完了しました。")

## 1. モデルの初期化と改良版線形化

In [ ]:
# モデルの読み込みと初期化
config_path = '../config/parameters.json'
baseline_model = load_model(config_path)

# 定常状態の計算
print("定常状態を計算中...")
baseline_ss = baseline_model.compute_steady_state()
print("定常状態の計算が完了しました。")

# 改良版線形化モデルの作成
linear_model = ImprovedLinearizedDSGE(baseline_model, baseline_ss)
linear_model.build_system_matrices()
P, Q = linear_model.solve_klein()

print("\nKlein法による線形化が完了しました。")
print(f"政策関数行列 P の次元: {P.shape}")
print(f"遷移行列 Q の次元: {Q.shape}")

## 2. 各種ショックに対するインパルス応答関数

In [ ]:
# インパルス応答関数の計算と可視化
shock_types = ['tfp', 'gov_spending', 'monetary', 'consumption_tax']
key_variables = ['Y', 'C', 'I', 'L', 'pi', 'T']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.ravel()

for i, shock in enumerate(shock_types):
    # インパルス応答の計算
    if 'tax' in shock:
        irf = linear_model.compute_impulse_response(shock, shock_size=1.0, periods=40)
    else:
        irf = linear_model.compute_impulse_response(shock, shock_size=1.0, periods=40)
    
    # GDPの応答をプロット
    ax = axes[i]
    ax.plot(irf.index, irf['Y'], linewidth=2, label='GDP')
    ax.plot(irf.index, irf['C'], linewidth=2, label='Consumption')
    ax.plot(irf.index, irf['I'], linewidth=2, label='Investment')
    
    ax.axhline(y=0, color='k', linestyle='--', alpha=0.5)
    ax.set_title(f'Response to {shock.replace("_", " ").title()} Shock')
    ax.set_xlabel('Quarters')
    ax.set_ylabel('% Deviation from SS')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. 税制改革シミュレーション with 移行動態

In [ ]:
# Enhanced Tax Simulatorの初期化
tax_simulator = EnhancedTaxSimulator(baseline_model)

# 複数の税制改革シナリオを定義
reforms = [
    TaxReform(
        name="消費税5%引き上げ（恒久的）",
        tau_c=0.15,  # 10% → 15%
        implementation='permanent'
    ),
    TaxReform(
        name="消費税5%引き上げ（段階的）",
        tau_c=0.15,
        implementation='phased',
        phase_in_periods=8  # 2年かけて段階的に実施
    ),
    TaxReform(
        name="所得税5%引き下げ",
        tau_l=0.15,  # 20% → 15%
        implementation='permanent'
    ),
    TaxReform(
        name="税制中立改革（消費税↑所得税↓）",
        tau_c=0.15,
        tau_l=0.15,
        implementation='permanent'
    )
]

In [ ]:
# 各改革のシミュレーション実行
simulation_results = {}

for reform in reforms:
    print(f"\nシミュレーション実行中: {reform.name}")
    results = tax_simulator.simulate_reform(reform, periods=100, compute_welfare=True)
    simulation_results[reform.name] = results
    
    # 結果サマリーの表示
    print(f"  厚生変化: {results.welfare_change:+.3f}%")
    print(f"  移行期間: {results.transition_periods} 四半期")
    print(f"  長期GDP効果: {(results.steady_state_reform.Y - results.steady_state_baseline.Y) / results.steady_state_baseline.Y * 100:+.3f}%")

## 4. 移行動態の可視化

In [ ]:
# 消費税改革の移行動態を詳細に分析
reform_name = "消費税5%引き上げ（恒久的）"
results = simulation_results[reform_name]

variables_to_plot = ['Y', 'C', 'I', 'L', 'T', 'B']
fig = tax_simulator.plot_transition_dynamics(
    results,
    variables=variables_to_plot,
    figsize=(14, 10)
)
plt.show()

In [ ]:
# 段階的実施vs即時実施の比較
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.ravel()

permanent_results = simulation_results["消費税5%引き上げ（恒久的）"]
phased_results = simulation_results["消費税5%引き上げ（段階的）"]

variables = ['Y', 'C', 'T', 'B']
titles = ['GDP', 'Consumption', 'Tax Revenue', 'Government Debt']

for i, (var, title) in enumerate(zip(variables, titles)):
    ax = axes[i]
    
    # 最初の40期間のみプロット
    periods = 40
    ax.plot(permanent_results.reform_path[var].iloc[:periods], 
            label='Immediate', linewidth=2, color='red')
    ax.plot(phased_results.reform_path[var].iloc[:periods], 
            label='Phased (8 quarters)', linewidth=2, color='blue', linestyle='--')
    ax.plot(permanent_results.baseline_path[var].iloc[:periods], 
            label='Baseline', linewidth=1, color='gray', alpha=0.7)
    
    ax.set_title(title)
    ax.set_xlabel('Quarters')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('Immediate vs Phased Implementation of 5pp Consumption Tax Increase', fontsize=14)
plt.tight_layout()
plt.show()

## 5. 税制改革の比較分析

In [ ]:
# 改革シナリオの比較表
comparison_df = tax_simulator.compare_reforms(
    reform_list=reforms,
    periods=100,
    variables=['Y', 'C', 'I', 'L', 'T', 'T/Y', 'Welfare']
)

# 結果をフォーマットして表示
styled_df = comparison_df.style.format({
    'Y': '{:+.2f}%',
    'C': '{:+.2f}%',
    'I': '{:+.2f}%',
    'L': '{:+.2f}%',
    'T': '{:+.2f}%',
    'T/Y': '{:+.2f}%',
    'Welfare': '{:+.3f}%',
    'Transition (quarters)': '{:.0f}'
})

# カラーマップの適用
styled_df = styled_df.background_gradient(cmap='RdYlGn', center=0, 
                                         subset=['Y', 'C', 'I', 'Welfare'])

display(HTML("<h3>税制改革の比較分析（定常状態での変化）</h3>"))
display(styled_df)

## 6. 財政への影響の詳細分析

In [ ]:
# 税制中立改革の財政影響を詳細分析
neutral_reform_results = simulation_results["税制中立改革（消費税↑所得税↓）"]

# 財政影響の表示
fiscal_impact = neutral_reform_results.fiscal_impact
print("税制中立改革の財政への影響")
print("=" * 60)

for horizon in fiscal_impact.index.get_level_values(0).unique():
    print(f"\n{horizon}:")
    horizon_data = fiscal_impact.loc[horizon]
    
    # 主要指標のみ表示
    key_vars = ['Y', 'T', 'T/Y ratio']
    for var in key_vars:
        if var in horizon_data.index:
            data = horizon_data.loc[var]
            print(f"  {var:12s}: Baseline={data['Baseline']:8.3f}, "
                  f"Reform={data['Reform']:8.3f}, "
                  f"Change={data['% Change']:+6.2f}%")

In [ ]:
# 各税収源の推移
fig, axes = plt.subplots(2, 2, figsize=(12, 8))
axes = axes.ravel()

tax_vars = ['Tc', 'Tl', 'Tk', 'Tf']
tax_names = ['Consumption Tax', 'Labor Income Tax', 'Capital Income Tax', 'Corporate Tax']

for i, (var, name) in enumerate(zip(tax_vars, tax_names)):
    ax = axes[i]
    
    # ベースラインと改革後の比較
    baseline = neutral_reform_results.baseline_path[var].iloc[:40]
    reform = neutral_reform_results.reform_path[var].iloc[:40]
    
    ax.plot(baseline.index, baseline, label='Baseline', linewidth=2)
    ax.plot(reform.index, reform, label='After Reform', linewidth=2, linestyle='--')
    
    # 変化率を右軸に表示
    ax2 = ax.twinx()
    change_pct = (reform - baseline) / baseline * 100
    ax2.plot(change_pct.index, change_pct, color='red', alpha=0.5, linewidth=1)
    ax2.set_ylabel('% Change', color='red')
    ax2.tick_params(axis='y', labelcolor='red')
    
    ax.set_title(name)
    ax.set_xlabel('Quarters')
    ax.set_ylabel('Revenue')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.suptitle('Tax Revenue by Source: Neutral Reform Impact', fontsize=14)
plt.tight_layout()
plt.show()

## 7. 分散分解分析

In [ ]:
# 予測誤差分散分解
variance_decomp = linear_model.variance_decomposition(
    periods=40,
    variables=['Y', 'C', 'I', 'pi', 'T']
)

# ヒートマップでの可視化
plt.figure(figsize=(10, 6))
sns.heatmap(variance_decomp, annot=True, fmt='.1f', cmap='YlOrRd', 
            cbar_kws={'label': 'Contribution (%)'})
plt.title('Forecast Error Variance Decomposition (40 quarters ahead)')
plt.xlabel('Shock Type')
plt.ylabel('Variable')
plt.tight_layout()
plt.show()

# 主要な発見の要約
print("\n分散分解の主要な発見:")
for var in variance_decomp.index:
    main_driver = variance_decomp.loc[var].idxmax()
    contribution = variance_decomp.loc[var, main_driver]
    print(f"- {var}の変動の{contribution:.1f}%は{main_driver}ショックによって説明される")

## 8. 最適税制の探索

In [ ]:
# 税収目標を設定（GDPの20%）
target_revenue = 0.20

# 税率の制約
tax_bounds = {
    'tau_c': (0.05, 0.25),  # 消費税: 5%-25%
    'tau_l': (0.10, 0.40),  # 所得税: 10%-40%
    'tau_f': (0.20, 0.40)   # 法人税: 20%-40%
}

print(f"目標税収: GDP比{target_revenue:.1%}")
print("\n厚生を最大化する最適税制を探索中...")

# 最適化の実行
optimal_welfare = tax_simulator.optimal_tax_mix(
    target_revenue=target_revenue,
    tax_bounds=tax_bounds,
    objective='welfare'
)

if optimal_welfare:
    print("\n厚生最大化の最適税率:")
    print(f"  消費税: {optimal_welfare['tau_c']:.1%}")
    print(f"  所得税: {optimal_welfare['tau_l']:.1%}")
    print(f"  法人税: {optimal_welfare['tau_f']:.1%}")

print("\nGDPを最大化する最適税制を探索中...")

# GDP最大化
optimal_output = tax_simulator.optimal_tax_mix(
    target_revenue=target_revenue,
    tax_bounds=tax_bounds,
    objective='output'
)

if optimal_output:
    print("\nGDP最大化の最適税率:")
    print(f"  消費税: {optimal_output['tau_c']:.1%}")
    print(f"  所得税: {optimal_output['tau_l']:.1%}")
    print(f"  法人税: {optimal_output['tau_f']:.1%}")

## 9. 結果のエクスポートとレポート作成

In [ ]:
# 主要な結果をDataFrameにまとめる
summary_results = pd.DataFrame()

for reform_name, results in simulation_results.items():
    summary = {
        'Reform': reform_name,
        'Welfare Change (%)': results.welfare_change,
        'Long-run GDP (%)': (results.steady_state_reform.Y - results.steady_state_baseline.Y) / results.steady_state_baseline.Y * 100,
        'Long-run C (%)': (results.steady_state_reform.C - results.steady_state_baseline.C) / results.steady_state_baseline.C * 100,
        'Long-run T/Y': results.steady_state_reform.T / results.steady_state_reform.Y,
        'Transition Period': results.transition_periods
    }
    summary_results = pd.concat([summary_results, pd.DataFrame([summary])], ignore_index=True)

# CSVとして保存
summary_results.to_csv('../results/advanced_simulation_summary.csv', index=False)
print("結果サマリーを保存しました: ../results/advanced_simulation_summary.csv")

# 詳細レポートの生成
for reform_name, results in simulation_results.items():
    safe_name = reform_name.replace('（', '_').replace('）', '').replace('↑', 'up').replace('↓', 'down')
    report_file = f'../results/report_{safe_name}.txt'
    tax_simulator.generate_report(results, report_file)
    print(f"詳細レポートを生成: {report_file}")

## まとめと政策提言

### 主要な発見

1. **消費税引き上げの影響**
   - 短期的には消費を抑制するが、長期的には財政の持続可能性を改善
   - 段階的実施は即時実施と比べて移行コストを軽減

2. **税制中立改革の有効性**
   - 消費税増税と所得税減税の組み合わせは、税収を維持しながら労働供給を促進
   - 厚生損失を最小限に抑えつつ、財政目標を達成可能

3. **最適税制の特徴**
   - 厚生最大化とGDP最大化では異なる税制構造が最適
   - 効率性と公平性のトレードオフを考慮した税制設計が重要

### 政策提言

1. 税制改革は段階的に実施し、経済主体の調整コストを軽減すべき
2. 単一税目の変更よりも、複数税目の組み合わせ改革が効果的
3. 長期的な財政持続可能性と短期的な経済への影響のバランスを考慮
4. 改革の実施時期は景気循環を考慮して決定すべき